In [1]:
 # Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json


In [2]:
crimes_df = pd.read_excel('2020crimes.xlsx')

crimes_df.head()

,Incident,Occurrence\nDate,Occurrence\nHour,NIBRS\nClass,NIBRSDescription,Offense\nCount,Beat,Premise,Block Range,StreetName,Street\nType,Suffix,City,ZIP Code
0,8220,2020-01-01,0,23G,Theft of motor vehicle parts or accessory,1,8C50,"Residence, Home (Includes Apartment)",9311,BELLA PINE,CT,NaN,HOUSTON,77078
1,18920,2020-01-01,0,13A,Aggravated Assault,1,17E30,"Residence, Home (Includes Apartment)",8701,GUSTINE,LN,NaN,HOUSTON,77031
2,23020,2020-01-01,0,290,"Destruction, damage, vandalism",1,11H30,"Residence, Home (Includes Apartment)",8064,LENORE,NaN,NaN,HOUSTON,77017
3,24120,2020-01-01,0,13B,Simple assault,1,17E10,"Residence, Home (Includes Apartment)",5930,DASHWOOD,DR,NaN,HOUSTON,77081
4,27120,2020-01-01,0,290,"Destruction, damage, vandalism",1,14D30,"Residence, Home (Includes Apartment)",5218,KENILWOOD,DR,NaN,HOUSTON,77033


In [3]:
crimes_df['Street\nType'] = crimes_df['Street\nType'].replace(np.nan, '', regex=True)

crimes_df = crimes_df.astype(str)

crimes_df

,Incident,Occurrence\nDate,Occurrence\nHour,NIBRS\nClass,NIBRSDescription,Offense\nCount,Beat,Premise,Block Range,StreetName,Street\nType,Suffix,City,ZIP Code
0,8220,2020-01-01,0,23G,Theft of motor vehicle parts or accessory,1,8C50,"Residence, Home (Includes Apartment)",9311,BELLA PINE,CT,nan,HOUSTON,77078
1,18920,2020-01-01,0,13A,Aggravated Assault,1,17E30,"Residence, Home (Includes Apartment)",8701,GUSTINE,LN,nan,HOUSTON,77031
2,23020,2020-01-01,0,290,"Destruction, damage, vandalism",1,11H30,"Residence, Home (Includes Apartment)",8064,LENORE,,nan,HOUSTON,77017
3,24120,2020-01-01,0,13B,Simple assault,1,17E10,"Residence, Home (Includes Apartment)",5930,DASHWOOD,DR,nan,HOUSTON,77081
4,27120,2020-01-01,0,290,"Destruction, damage, vandalism",1,14D30,"Residence, Home (Includes Apartment)",5218,KENILWOOD,DR,nan,HOUSTON,77033
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86909,72754920,2020-05-31,23,23G,Theft of motor vehicle parts or accessory,1,22B20,"Parking Lot, Garage",16501,CENTRAL GREEN,BLVD,nan,HOUSTON,77032
86910,72844720,2020-05-31,23,23F,Theft from motor vehicle,1,22B10,"Parking Lot, Garage",7815,WILLOW CHASE,BLVD,nan,HOUSTON,77070
86911,72913020,2020-05-31,23,13C,Intimidation,1,18F40,"Residence, Home (Includes Apartment)",9550,ELLA LEE,LN,nan,HOUSTON,77063
86912,72943020,2020-05-31,23,240,Motor vehicle theft,1,15E40,"Parking Lot, Garage",10000,FANNIN,ST,nan,HOUSTON,77045


In [4]:
# crimes_df['address_number'] = crimes_df['Block Range'].str.split('-').str[0]
crimes_df['address_number'] = crimes_df['Block Range'].apply(lambda x: x.split('-')[0])

In [5]:
crimes_df['address'] = crimes_df['address_number'] + ' ' + crimes_df['StreetName'] + ' ' + crimes_df['Street\nType'] + ', ' + crimes_df['City'] + ', TX'

In [6]:
address_list = crimes_df['address'].to_list()

crimes_df

,Incident,Occurrence\nDate,Occurrence\nHour,NIBRS\nClass,NIBRSDescription,Offense\nCount,Beat,Premise,Block Range,StreetName,Street\nType,Suffix,City,ZIP Code,address_number,address
0,8220,2020-01-01,0,23G,Theft of motor vehicle parts or accessory,1,8C50,"Residence, Home (Includes Apartment)",9311,BELLA PINE,CT,nan,HOUSTON,77078,9311,"9311 BELLA PINE CT, HOUSTON, TX"
1,18920,2020-01-01,0,13A,Aggravated Assault,1,17E30,"Residence, Home (Includes Apartment)",8701,GUSTINE,LN,nan,HOUSTON,77031,8701,"8701 GUSTINE LN, HOUSTON, TX"
2,23020,2020-01-01,0,290,"Destruction, damage, vandalism",1,11H30,"Residence, Home (Includes Apartment)",8064,LENORE,,nan,HOUSTON,77017,8064,"8064 LENORE , HOUSTON, TX"
3,24120,2020-01-01,0,13B,Simple assault,1,17E10,"Residence, Home (Includes Apartment)",5930,DASHWOOD,DR,nan,HOUSTON,77081,5930,"5930 DASHWOOD DR, HOUSTON, TX"
4,27120,2020-01-01,0,290,"Destruction, damage, vandalism",1,14D30,"Residence, Home (Includes Apartment)",5218,KENILWOOD,DR,nan,HOUSTON,77033,5218,"5218 KENILWOOD DR, HOUSTON, TX"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86909,72754920,2020-05-31,23,23G,Theft of motor vehicle parts or accessory,1,22B20,"Parking Lot, Garage",16501,CENTRAL GREEN,BLVD,nan,HOUSTON,77032,16501,"16501 CENTRAL GREEN BLVD, HOUSTON, TX"
86910,72844720,2020-05-31,23,23F,Theft from motor vehicle,1,22B10,"Parking Lot, Garage",7815,WILLOW CHASE,BLVD,nan,HOUSTON,77070,7815,"7815 WILLOW CHASE BLVD, HOUSTON, TX"
86911,72913020,2020-05-31,23,13C,Intimidation,1,18F40,"Residence, Home (Includes Apartment)",9550,ELLA LEE,LN,nan,HOUSTON,77063,9550,"9550 ELLA LEE LN, HOUSTON, TX"
86912,72943020,2020-05-31,23,240,Motor vehicle theft,1,15E40,"Parking Lot, Garage",10000,FANNIN,ST,nan,HOUSTON,77045,10000,"10000 FANNIN ST, HOUSTON, TX"


In [ ]:
# import censusgeocode as cg

# crime_lats = []
# crime_lngs = []
# census_tract_crime = []
# count = 1

# for address in address_list:
#     try:
#         this_address = cg.onelineaddress(address)
#         crime_lats.append(this_address[0]['coordinates']['y'])
#         crime_lngs.append(this_address[0]['coordinates']['x'])
#         census_tract_crime.append(this_address[0]['geographies']['2010 Census Blocks'][0]['TRACT'])
#         print(f'{count} - success')
#         count += 1
#     except:
#         crime_lats.append('NaN')
#         crime_lngs.append('NaN')
#         census_tract_crime.append('NaN') 
#         print(f'{count} - fail')
#         count += 1

1 - success
2 - success
3 - success
4 - success
5 - success
6 - success
7 - fail
8 - success
9 - success
10 - success
11 - success
12 - success
13 - success
14 - success
15 - success
16 - success
17 - success
18 - success
19 - success
20 - success
21 - success
22 - success
23 - success
24 - success
25 - fail
26 - success
27 - fail
28 - success
29 - success
30 - success
31 - success
32 - success
33 - success
34 - success
35 - success
36 - success
37 - success
38 - success
39 - success
40 - success
41 - success
42 - fail
43 - success
44 - success
45 - success
46 - success
47 - success
48 - success
49 - success
50 - success
51 - success
52 - success
53 - success
54 - fail
55 - success
56 - fail
57 - fail
58 - success
59 - success
60 - success
61 - success
62 - success
63 - success
64 - success
65 - success
66 - success
67 - success
68 - success
69 - success
70 - success
71 - success
72 - success
73 - success
74 - success
75 - success
76 - success
77 - success
78 - success
79 - success
80 

609 - success
610 - success
611 - success
612 - success
613 - success
614 - fail
615 - fail
616 - success
617 - success
618 - success
619 - fail
620 - success
621 - fail
622 - success
623 - success
624 - success
625 - fail
626 - fail
627 - success
628 - success
629 - success
630 - success
631 - success
632 - success
633 - success
634 - success
635 - success
636 - success
637 - fail
638 - success
639 - success
640 - success
641 - fail
642 - success
643 - success
644 - success
645 - fail
646 - fail
647 - success
648 - success
649 - success
650 - success
651 - success
652 - success
653 - fail
654 - success
655 - success
656 - success
657 - success
658 - success
659 - success
660 - fail
661 - fail
662 - success
663 - fail
664 - success
665 - success
666 - success
667 - success
668 - success
669 - success
670 - success
671 - fail
672 - success
673 - success
674 - success
675 - success
676 - success
677 - success
678 - success
679 - success
680 - success
681 - success
682 - fail
683 - succes

1203 - fail
1204 - success
1205 - success
1206 - success
1207 - success
1208 - success
1209 - success
1210 - success
1211 - success
1212 - success
1213 - success
1214 - success
1215 - success
1216 - success
1217 - fail
1218 - success
1219 - success
1220 - success
1221 - success
1222 - success
1223 - success
1224 - success
1225 - success
1226 - success
1227 - success
1228 - success
1229 - success
1230 - success
1231 - success
1232 - success
1233 - success
1234 - success
1235 - success
1236 - success
1237 - success
1238 - success
1239 - success
1240 - success
1241 - success
1242 - success
1243 - success
1244 - success
1245 - success
1246 - success
1247 - success
1248 - success
1249 - success
1250 - success
1251 - success
1252 - success
1253 - fail
1254 - success
1255 - success
1256 - success
1257 - success
1258 - success
1259 - fail
1260 - success
1261 - success
1262 - success
1263 - success
1264 - success
1265 - success
1266 - success
1267 - success
1268 - success
1269 - fail
1270 - suc

1766 - success
1767 - success
1768 - fail
1769 - success
1770 - success
1771 - success
1772 - success
1773 - success
1774 - success
1775 - success
1776 - success
1777 - success
1778 - success
1779 - fail
1780 - success
1781 - success
1782 - success
1783 - success
1784 - success
1785 - success
1786 - success
1787 - success
1788 - success
1789 - success
1790 - success
1791 - success
1792 - success
1793 - success
1794 - success
1795 - fail
1796 - fail
1797 - fail
1798 - success
1799 - fail
1800 - success
1801 - fail
1802 - fail
1803 - success
1804 - success
1805 - success
1806 - success
1807 - success
1808 - success
1809 - success
1810 - success
1811 - success
1812 - success
1813 - success
1814 - success
1815 - success
1816 - success
1817 - success
1818 - success
1819 - success
1820 - success
1821 - success
1822 - success
1823 - success
1824 - success
1825 - success
1826 - fail
1827 - success
1828 - success
1829 - success
1830 - success
1831 - success
1832 - success
1833 - success
1834 - 

In [10]:
# # test_df = crimes_df[0:10]

# test_df['Lat'] = crime_lats
# test_df['Lng'] = crime_lngs
# test_df['Census_Tract'] = census_tract_crime

# # test_df = test_df[['NIBRSDescription', 'Occurrence\nDate', 'Occurrence\nHour', 'address', 'Lat', 'Lng', 'Census_Tract']]
# # test_df.head(10)

/Users/ryanashcraft/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/ryanashcraft/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/Users/ryanashcraft/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying 

,NIBRSDescription,Occurrence\nDate,Occurrence\nHour,address,Lat,Lng,Census_Tract
0,Theft of motor vehicle parts or accessory,2020-01-01,0,"9311 BELLA PINE CT, HOUSTON, TX",29.8562,-95.2523,231200
1,Aggravated Assault,2020-01-01,0,"8701 GUSTINE LN, HOUSTON, TX",29.6696,-95.5305,423202
2,"Destruction, damage, vandalism",2020-01-01,0,"8064 LENORE , HOUSTON, TX",29.6765,-95.2635,320602
3,Simple assault,2020-01-01,0,"5930 DASHWOOD DR, HOUSTON, TX",29.7074,-95.4858,421403
4,"Destruction, damage, vandalism",2020-01-01,0,"5218 KENILWOOD DR, HOUSTON, TX",29.6726,-95.344,332000
5,"Destruction, damage, vandalism",2020-01-01,0,"2851 WALLINGFORD DR, HOUSTON, TX",29.7325,-95.5638,452201
6,"Destruction, damage, vandalism",2020-01-01,0,"16410 SKEET CT, HOUSTON, TX",NaN,NaN,NaN
7,Theft from motor vehicle,2020-01-01,0,"400 80TH ST, HOUSTON, TX",29.7314,-95.2815,311100
8,Simple assault,2020-01-01,0,"2434 LIDSTONE , HOUSTON, TX",29.7157,-95.3217,311900
9,Simple assault,2020-01-01,0,"8115 PENROD , HOUSTON, TX",29.8258,-95.2783,230900


In [ ]:
from geopy.geocoders import Nominatim

crime_lats = []
crime_lngs = []

for address in address_list:
    try:
        geolocator = Nominatim(user_agent="DreamTeam")
        location = geolocator.geocode(address)
#         print(location.raw)
        print(f'{address} LatLng:({location.latitude, location.longitude})')
        crime_lats.append(location.latitude)
        crime_lngs.append(location.longitude)
    except:
        print('Not Found! :(')
        crime_lats.append('NaN')
        crime_lngs.append('NaN')

9311 BELLA PINE CT, HOUSTON, TX LatLng:((29.856284897959185, -95.25162830612244))
8701 GUSTINE LN, HOUSTON, TX LatLng:((29.66944225726196, -95.53262835232545))
8064 LENORE , HOUSTON, TX LatLng:((29.676494956905984, -95.26344950122187))
5930 DASHWOOD DR, HOUSTON, TX LatLng:((29.707436610117348, -95.48575013163966))
5218 KENILWOOD DR, HOUSTON, TX LatLng:((29.672524398096606, -95.3440337145399))
2851 WALLINGFORD DR, HOUSTON, TX LatLng:((29.732533090109193, -95.56373508028449))
16410 SKEET CT, HOUSTON, TX LatLng:((29.588749651908888, -95.51077832154154))
Not Found! :(
2434 LIDSTONE , HOUSTON, TX LatLng:((29.71563288888889, -95.32175177777778))
8115 PENROD , HOUSTON, TX LatLng:((29.825890714285716, -95.27823342857143))
Not Found! :(
1415 GREENS PKWY, HOUSTON, TX LatLng:((29.94285753624613, -95.44339853624614))
3588 OCEE ST, HOUSTON, TX LatLng:((29.724780484949836, -95.5292034013378))
5425 NIELAN , HOUSTON, TX LatLng:((29.810301040816327, -95.27928628571428))
228 ALTIC ST, HOUSTON, TX LatLng

4401 HARRISBURG BLVD, HOUSTON, TX LatLng:((29.744261, -95.332169))
9801 MEADOWGLEN LN, HOUSTON, TX LatLng:((29.732204, -95.543416))
8134 OAK KNOLL LN, HOUSTON, TX LatLng:((29.839696, -95.280324))
1950 SPENWICK DR, HOUSTON, TX LatLng:((29.8064826122449, -95.48907093877551))
Not Found! :(
3230 GESSNER RD, HOUSTON, TX LatLng:((29.8946776, -95.5436183))
11757 BRIAR FOREST DR, HOUSTON, TX LatLng:((29.747109611179162, -95.58571844471666))
7935 ELLINGER LN, HOUSTON, TX LatLng:((29.850052291666668, -95.51196837500001))
13930 FONDREN RD, HOUSTON, TX LatLng:((29.619366, -95.5092476))
2314 GENTRYSIDE DR, HOUSTON, TX LatLng:((29.737918786347013, -95.61788345529926))
6627 SOUTHWEST FWY, HOUSTON, TX LatLng:((29.6553136, -95.5638796))
11939 BISSONNET ST, HOUSTON, TX LatLng:((29.675173469387754, -95.58786632653062))
Not Found! :(
17111 TOMBALL PKWY, HOUSTON, TX LatLng:((29.9455647, -95.5274958))
7525 SHERMAN , HOUSTON, TX LatLng:((29.734307489795917, -95.29078412244898))
4522 AVERILL ST, HOUSTON, TX L

Not Found! :(
742 SAM HOUSTON PKWY, HOUSTON, TX LatLng:((30.2908453, -94.8467702))
742 SAM HOUSTON PKWY, HOUSTON, TX LatLng:((30.2908453, -94.8467702))
3300 YELLOWSTONE BLVD, HOUSTON, TX LatLng:((29.697377, -95.374383))
2100 SHADOWDALE DR, HOUSTON, TX LatLng:((29.80942, -95.554281))
8972 IMOGENE ST, HOUSTON, TX LatLng:((29.685228771550094, -95.53782230385087))
9200 CULLEN BLVD, HOUSTON, TX LatLng:((29.702756, -95.348271))
4923 DANFIELD DR, HOUSTON, TX LatLng:((29.59990711111111, -95.45330222222222))
12261 FONDREN RD, HOUSTON, TX LatLng:((29.6256184, -95.5074302))
Not Found! :(
900 ROPER ST, HOUSTON, TX LatLng:((29.648483469798656, -95.23608283221476))
4910 SUNBEAM ST, HOUSTON, TX LatLng:((29.650532470588235, -95.3512461764706))
5500 ANTOINE DR, HOUSTON, TX LatLng:((29.846794, -95.473138))
1 GOODSON DR, HOUSTON, TX LatLng:((29.925221, -95.407036))
6010 SOUTHLEA ST, HOUSTON, TX LatLng:((29.68620781357722, -95.32662069570178))
2600 GRAMERCY ST, HOUSTON, TX LatLng:((29.70448, -95.418411))


Not Found! :(


In [32]:
# print(lat_lngs[0:100])

[(29.8476134, -95.348289), (29.9868335, -95.295932), (29.98473266366446, -95.30688860932969), (29.657766, -95.278452), (29.657766, -95.278452), (31.5607982, -95.4899006), (29.657766, -95.278452), 'NaN', (29.657766, -95.278452), (30.025436, -95.3039301), 'NaN', (29.7376895, -95.4882776), (30.0405146, -95.2340988), 'NaN', (30.022246552100682, -95.23211342705905), (30.0497077, -95.1846301), (30.05796, -95.21781), (30.0686045, -95.2332297), 'NaN', 'NaN', 'NaN', (30.042981962962962, -95.22070011111111), (30.0684, -95.218287), (30.0497077, -95.1846301), (30.03565694949495, -95.24691032323233), (30.02610416372933, -95.24344966936289), (30.0497077, -95.1846301), (34.4416107, -118.477689), (30.051843, -95.2469618), 'NaN', (30.07984667073171, -95.17923157317072), 'NaN', (30.045477, -95.224671), 'NaN', 'NaN', (30.025531, -95.241185), 'NaN', (30.04311, -95.196724), (30.046943, -95.184499), (30.0448554, -95.1828603), (30.046943, -95.184499), (30.044215, -95.216505), (30.083738, -95.19146), 'NaN', (

In [2]:
police_df = pd.read_excel('police_stations.xlsx')

police_df.head()

,Station,Address,Beat
0,Central,"33 Artesian, Houston, TX",2A40
1,Downtown,"1900 Rusk, Houston, TX",1A10
2,North,"9455 West Montgomery, Houston, TX",6B40
3,North Belt,"100 Glenborough, Houston, TX",22B10
4,Northwest,"6000 Teague, Houston, TX",5F30


In [3]:
police_df['City'] = [x.split(',')[1] for x in police_df['Address']]
police_df

,Station,Address,Beat,City
0,Central,"33 Artesian, Houston, TX",2A40,Houston
1,Downtown,"1900 Rusk, Houston, TX",1A10,Houston
2,North,"9455 West Montgomery, Houston, TX",6B40,Houston
3,North Belt,"100 Glenborough, Houston, TX",22B10,Houston
4,Northwest,"6000 Teague, Houston, TX",5F30,Houston
5,Northeast,"8301 Ley Road, Houston, TX",8C30,Houston
6,South Central,"2202 St. Emanuel, Houston, TX",10H50,Houston
7,Eastside,"7525 Sherman, Houston, TX",11H10,Houston
8,Clear Lake,"2855 Bay Area Blvd, Houston, TX",12D50,Houston
9,Southeast,"8300 Mykawa, Houston, TX",14D30,Houston


In [4]:
police_df['State'] = [x.split(',')[2] for x in police_df['Address']]
police_df

,Station,Address,Beat,City,State
0,Central,"33 Artesian, Houston, TX",2A40,Houston,TX
1,Downtown,"1900 Rusk, Houston, TX",1A10,Houston,TX
2,North,"9455 West Montgomery, Houston, TX",6B40,Houston,TX
3,North Belt,"100 Glenborough, Houston, TX",22B10,Houston,TX
4,Northwest,"6000 Teague, Houston, TX",5F30,Houston,TX
5,Northeast,"8301 Ley Road, Houston, TX",8C30,Houston,TX
6,South Central,"2202 St. Emanuel, Houston, TX",10H50,Houston,TX
7,Eastside,"7525 Sherman, Houston, TX",11H10,Houston,TX
8,Clear Lake,"2855 Bay Area Blvd, Houston, TX",12D50,Houston,TX
9,Southeast,"8300 Mykawa, Houston, TX",14D30,Houston,TX


In [5]:
police_df['Address'] = [x.split(',')[0] for x in police_df['Address']]
police_df

,Station,Address,Beat,City,State
0,Central,33 Artesian,2A40,Houston,TX
1,Downtown,1900 Rusk,1A10,Houston,TX
2,North,9455 West Montgomery,6B40,Houston,TX
3,North Belt,100 Glenborough,22B10,Houston,TX
4,Northwest,6000 Teague,5F30,Houston,TX
5,Northeast,8301 Ley Road,8C30,Houston,TX
6,South Central,2202 St. Emanuel,10H50,Houston,TX
7,Eastside,7525 Sherman,11H10,Houston,TX
8,Clear Lake,2855 Bay Area Blvd,12D50,Houston,TX
9,Southeast,8300 Mykawa,14D30,Houston,TX


In [6]:
# Convert Integer to Specific Beat
police_df.loc[police_df.Beat == 170000000000, 'Beat'] = '17E10'

police_df

,Station,Address,Beat,City,State
0,Central,33 Artesian,2A40,Houston,TX
1,Downtown,1900 Rusk,1A10,Houston,TX
2,North,9455 West Montgomery,6B40,Houston,TX
3,North Belt,100 Glenborough,22B10,Houston,TX
4,Northwest,6000 Teague,5F30,Houston,TX
5,Northeast,8301 Ley Road,8C30,Houston,TX
6,South Central,2202 St. Emanuel,10H50,Houston,TX
7,Eastside,7525 Sherman,11H10,Houston,TX
8,Clear Lake,2855 Bay Area Blvd,12D50,Houston,TX
9,Southeast,8300 Mykawa,14D30,Houston,TX


In [7]:
police_df['Address'] = police_df['Address'].replace(' ', '+', regex=True)

police_df

,Station,Address,Beat,City,State
0,Central,33+Artesian,2A40,Houston,TX
1,Downtown,1900+Rusk,1A10,Houston,TX
2,North,9455+West+Montgomery,6B40,Houston,TX
3,North Belt,100+Glenborough,22B10,Houston,TX
4,Northwest,6000+Teague,5F30,Houston,TX
5,Northeast,8301+Ley+Road,8C30,Houston,TX
6,South Central,2202+St.+Emanuel,10H50,Houston,TX
7,Eastside,7525+Sherman,11H10,Houston,TX
8,Clear Lake,2855+Bay+Area+Blvd,12D50,Houston,TX
9,Southeast,8300+Mykawa,14D30,Houston,TX


In [8]:
police_df.loc[police_df.Address == '6000+Teague', 'Address'] = '14421+Sommermeyer+St'
police_df.loc[police_df.Address == '8605+Westplace+Dr', 'Address'] = '11851+S+Gessner+Rd'
police_df.loc[police_df.Address == '6227+Southwest+Freeway', 'Address'] = '6227+Southwest+Fwy'
police_df.loc[police_df.Address == '3915+Rustic+Woods', 'Address'] = '3403+Rivers+Edge'

police_df

,Station,Address,Beat,City,State
0,Central,33+Artesian,2A40,Houston,TX
1,Downtown,1900+Rusk,1A10,Houston,TX
2,North,9455+West+Montgomery,6B40,Houston,TX
3,North Belt,100+Glenborough,22B10,Houston,TX
4,Northwest,14421+Sommermeyer+St,5F30,Houston,TX
5,Northeast,8301+Ley+Road,8C30,Houston,TX
6,South Central,2202+St.+Emanuel,10H50,Houston,TX
7,Eastside,7525+Sherman,11H10,Houston,TX
8,Clear Lake,2855+Bay+Area+Blvd,12D50,Houston,TX
9,Southeast,8300+Mykawa,14D30,Houston,TX


In [17]:
police_lats = []
police_lngs = []
census_tract = []

for street, city in zip(police_df['Address'], police_df['City']):
    try:
        census_url = f'https://geocoding.geo.census.gov/geocoder/geographies/address?street={street}&city={city}&state=TX&benchmark=Public_AR_Census2010&vintage=Census2010_Census2010&layers=14&format=json'
        response = requests.get(census_url)
        census_json = response.json()
    #     print(census_json)
        police_lats.append(census_json['result']['addressMatches'][0]['coordinates']['y'])
        police_lngs.append(census_json['result']['addressMatches'][0]['coordinates']['x'])
        census_tract.append(census_json['result']['addressMatches'][0]['geographies']['Census Blocks'][0]['TRACT'])
        print(f"{street} Latitude: {census_json['result']['addressMatches'][0]['coordinates']['y']}")
        print(f"{street} Longitude: {census_json['result']['addressMatches'][0]['coordinates']['x']}")
        print(f"{street} Census Tract: {census_json['result']['addressMatches'][0]['geographies']['Census Blocks'][0]['TRACT']}")
        print('-------------')                                        
    except:
        print(f'{street}: has failed you')


33+Artesian Latitude: 29.764551
33+Artesian Longitude: -95.36878
33+Artesian Census Tract: 510100
-------------
1900+Rusk Latitude: 29.753761
1900+Rusk Longitude: -95.35599
1900+Rusk Census Tract: 100000
-------------
9455+West+Montgomery Latitude: 29.88037
9455+West+Montgomery Longitude: -95.44648
9455+West+Montgomery Census Tract: 533100
-------------
100+Glenborough Latitude: 29.951307
100+Glenborough Longitude: -95.42141
100+Glenborough Census Tract: 550100
-------------
14421+Sommermeyer+St Latitude: 29.859278
14421+Sommermeyer+St Longitude: -95.54168
14421+Sommermeyer+St Census Tract: 521600
-------------
8301+Ley+Road Latitude: 29.831425
8301+Ley+Road Longitude: -95.27464
8301+Ley+Road Census Tract: 231000
-------------
2202+St.+Emanuel Latitude: 29.742254
2202+St.+Emanuel Longitude: -95.362625
2202+St.+Emanuel Census Tract: 312300
-------------
7525+Sherman Latitude: 29.73425
7525+Sherman Longitude: -95.290825
7525+Sherman Census Tract: 311100
-------------
2855+Bay+Area+Blvd L

In [18]:
police_df['Lat'] = police_lats
police_df['Lng'] = police_lngs
police_df['Census_Tract'] = census_tract

police_df

,Station,Address,Beat,City,State,Lat,Lng,Census_Tract
0,Central,33+Artesian,2A40,Houston,TX,29.764551,-95.368780,510100
1,Downtown,1900+Rusk,1A10,Houston,TX,29.753761,-95.355990,100000
2,North,9455+West+Montgomery,6B40,Houston,TX,29.880370,-95.446480,533100
3,North Belt,100+Glenborough,22B10,Houston,TX,29.951307,-95.421410,550100
4,Northwest,14421+Sommermeyer+St,5F30,Houston,TX,29.859278,-95.541680,521600
5,Northeast,8301+Ley+Road,8C30,Houston,TX,29.831425,-95.274640,231000
6,South Central,2202+St.+Emanuel,10H50,Houston,TX,29.742254,-95.362625,312300
7,Eastside,7525+Sherman,11H10,Houston,TX,29.734250,-95.290825,311100
8,Clear Lake,2855+Bay+Area+Blvd,12D50,Houston,TX,29.580847,-95.105050,340400
9,Southeast,8300+Mykawa,14D30,Houston,TX,29.656624,-95.315420,332400
